In [10]:
from selenium import webdriver
import time
import pandas as pd

In [30]:
#https://stackoverflow.com/questions/42478591/python-selenium-chrome-webdriver
#https://stackoverflow.com/questions/47074208/chrome-fails-to-load-with-python-selenium-chromedriver
opts = webdriver.ChromeOptions()
opts.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path=r"C:\Users\woote\Desktop\Python\Projects\chromedriver.exe",options=opts)

In [31]:
driver.get('https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/wroclaw/krzyki')

time.sleep(1)
try:
    accept = driver.find_element_by_id('onetrust-accept-btn-handler')
    accept.click()
except:
    pass

#More Filters
moreFilters = driver.find_element_by_id('search-form-more-filters')
moreFilters.click()
    
#Minimum area
areaMin = driver.find_element_by_id('areaMin')
areaMin.send_keys('45')
#Maximum area
areaMax = driver.find_element_by_id('areaMax')
areaMax.send_keys('65')
#Maximum Price per meter
maxPrice = driver.find_element_by_id('pricePerMeterMax')
maxPrice.send_keys('9000')
#Minimum Year
buildYear = driver.find_element_by_id('buildYearMin')
buildYear.send_keys('2010')

#Find locals with balconys and elevators
attributes = driver.find_element_by_xpath("//ul[contains(@id, 'extras')]")
for child in attributes.find_elements_by_xpath(".//*"):
    if child.get_attribute("class") == 'css-1xx0ttw elfgyn10':
        if child.text == 'Balkon' or child.text == 'Winda':
            child.click()
            
#Submit search
driver.find_element_by_id('search-form-submit').click()

In [32]:
#Move down so that needed page elements will become visible
driver.execute_script("window.scrollTo(0, 9000)")
time.sleep(1)
#Get the number of pages - it will be used later to click through pages
Buttons = driver.find_elements_by_xpath("//button[contains(@class, 'eoupkm71 css-1lc8b1f e11e36i3')]")
for i in range(len(Buttons)-1,0,-1):
    try:
        if int(Buttons[i].text):
            lastPage = int(Buttons[i].text)
            break
    except:
        pass

In [33]:
offersList = driver.find_elements_by_xpath("//a[contains(@class, 'css-jf4j3r es62z2j27')]")
links = [elem.get_attribute('href') for elem in offersList]

In [34]:
#Loop through pages. Open offers links in new tabs and then close them
linkS = []
titleS = []
cenaS = []
cenaM2S = []
rokBudowyS = []
powierzchniaS = []
rynekS = []
pokojeS = []
pietroS = []

for i in range(1,lastPage+1):
    
    offersList = driver.find_elements_by_xpath("//a[contains(@class, 'css-jf4j3r es62z2j27')]")
    links = [elem.get_attribute('href') for elem in offersList]

    for link in links:
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(link)
        try:
            cena = driver.find_element_by_xpath("//strong[@aria-label='Cena']")
            cenaM2 = driver.find_element_by_xpath("//div[@aria-label='Cena za metr kwadratowy']")
            rokBudowy = driver.find_element_by_xpath("//div[@aria-label='Rok budowy']").find_elements_by_tag_name('div')[1]
            powierzchnia = driver.find_element_by_xpath("//div[@aria-label='Powierzchnia']").find_elements_by_tag_name('div')[1]
            rynek = driver.find_element_by_xpath("//div[@aria-label='Rynek']").find_elements_by_tag_name('div')[1]
            pokoje = driver.find_element_by_xpath("//div[@aria-label='Liczba pokoi']").find_elements_by_tag_name('div')[1]
            pietro = driver.find_element_by_xpath("//div[@aria-label='Piętro']").find_elements_by_tag_name('div')[1]
            title = driver.find_element_by_xpath("//h1[contains(@class, 'css-11kn46p eu6swcv15')]")
        
            linkS.append(link) 
            titleS.append(title.text)
            cenaS.append(cena.text)
            cenaM2S.append(cenaM2.text)
            rokBudowyS.append(rokBudowy.text)
            powierzchniaS.append(powierzchnia.text)
            rynekS.append(rynek.text)
            pokojeS.append(pokoje.text)
            pietroS.append(pietro.text)
        
        except:
            continue
        
        time.sleep(1)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    
    driver.execute_script("window.scrollTo(0, 9000)")
    time.sleep(1)
    
    try:
        nextPage = driver.find_element_by_xpath("//button[@aria-label='następna strona']")
        nextPage.click()
    except Exception as e:
        try:
            driver.refresh()
        except:
            print(e.__doc__)
            break

In [37]:
Results = pd.DataFrame({'Hyperlink':linkS, 'Tytuł':titleS, 'Cena':cenaS, 'Cena M2':cenaM2S, 'Rok Budowy':rokBudowyS,\
                        'Powierzchnia':powierzchniaS, 'Rynek':rynekS,'Pokoje':pokojeS, 'Piętro':pietroS},\
                       columns=['Hyperlink','Tytuł','Cena','Cena M2','Rok Budowy','Powierzchnia','Rynek','Pokoje','Piętro'])
Results.drop_duplicates(inplace = True)

In [39]:
Results.to_excel('results.xlsx')